# Chapter 4 - Exercise 8: Integrated Bioprinting Process Design

## 🎯 CAPSTONE EXERCISE: Complete Process Optimization

## Learning Objectives
- Integrate all bioprinting concepts from Exercises 1-7
- Design a complete bioprinting process from concept to execution
- Balance competing requirements (cells, mechanics, transport, rheology)
- Optimize multi-parameter design spaces
- Predict success and identify failure modes
- Develop troubleshooting strategies

## Background

This **capstone exercise** integrates all concepts from Chapter 4:

1. **Cell Viability** (Ex 1-2): Shear stress, pressure, time, temperature
2. **Mass Transport** (Ex 3): Diffusion, perfusion, oxygen supply
3. **Gelation Kinetics** (Ex 4): Crosslinking mechanisms, gelation time
4. **Photopolymerization** (Ex 5): Light dose, penetration, cell safety
5. **Scaffold Architecture** (Ex 6): Porosity, pore size, mechanical properties
6. **Rheology** (Ex 7): Viscosity, shear-thinning, shape fidelity

### The Design Challenge:

**Design a bioprinting process for a 10×10×5 mm cardiac tissue construct containing cardiomyocytes (CMs) and endothelial cells (ECs).**

**Requirements:**
- Cell viability >80% at 24 hours
- Mechanical properties matching native myocardium (E = 10-50 kPa)
- Vascularization potential (pores >200 µm)
- Print time <2 hours
- Shape fidelity (±10% dimensional accuracy)

**Constraints:**
- Cell density: 10-50 million cells/mL
- Maximum shear stress: 5 kPa (cardiomyocytes are sensitive)
- Maximum pressure: 200 kPa
- Temperature: 15-37°C during printing
- Available materials: GelMA, alginate, collagen, fibrinogen

### Design Process:
1. Select bioprinting technique
2. Design bioink formulation
3. Optimize printing parameters
4. Predict outcomes
5. Identify risks and mitigation strategies

## Setup: Import All Functions from Previous Exercises

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install numpy matplotlib pandas seaborn scipy plotly -q

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize, fsolve
from scipy.integrate import odeint
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ All libraries loaded successfully!")
print("Ready for integrated bioprinting design!")

## Part 1: Comprehensive Design Framework Functions

In [ ]:
# ========================================
# INTEGRATED BIOPRINTING DESIGN FUNCTIONS
# ========================================

# From Exercise 1: Cell Viability
def predict_cell_viability_shear(shear_stress_kPa, exposure_time_s, cell_type='general'):
    """Predict cell viability from shear exposure."""
    # Cell-specific sensitivity
    if cell_type == 'cardiomyocyte':
        critical_stress = 3  # kPa
        sensitivity = 0.4
    elif cell_type == 'endothelial':
        critical_stress = 5  # kPa
        sensitivity = 0.3
    else:
        critical_stress = 4  # kPa
        sensitivity = 0.35
    
    # Damage accumulation model
    damage = sensitivity * (shear_stress_kPa / critical_stress) * np.sqrt(exposure_time_s)
    viability = 95 * np.exp(-damage)
    return np.clip(viability, 0, 100)

# From Exercise 3: Mass Transport
def calculate_oxygen_penetration(time_hours, D=2e-9):
    """Calculate oxygen diffusion distance."""
    t_s = time_hours * 3600
    L = np.sqrt(4 * D * t_s)
    return L * 1e6  # Convert to µm

def assess_vascularization_need(construct_thickness_mm, pore_size_um):
    """Assess if vascularization is needed."""
    critical_distance = 200  # µm
    thickness_um = construct_thickness_mm * 1000
    
    if thickness_um > 2 * critical_distance:
        if pore_size_um < 200:
            return {'needed': True, 'urgency': 'critical', 
                   'reason': 'Thick construct with small pores'}
        else:
            return {'needed': True, 'urgency': 'important',
                   'reason': 'Thick construct needs vessel ingrowth'}
    else:
        return {'needed': False, 'urgency': 'low',
               'reason': 'Thin enough for diffusion'}

# From Exercise 4: Gelation
def calculate_gelation_time(temperature_C, concentration_percent, mechanism='ionic'):
    """Estimate gelation time."""
    if mechanism == 'ionic':
        # Fast ionic crosslinking (alginate + Ca2+)
        base_time = 10  # seconds
        t_gel = base_time * (2 / concentration_percent) * (25 / temperature_C)
    elif mechanism == 'photo':
        # Photopolymerization
        base_time = 5  # seconds
        t_gel = base_time * (1 / concentration_percent)
    else:  # thermal
        # Temperature-sensitive (gelatin, Pluronic)
        base_time = 60  # seconds
        t_gel = base_time * np.exp(-(37 - temperature_C) / 5)
    
    return max(t_gel, 1)

# From Exercise 6: Scaffold Mechanics
def gibson_ashby_modulus(E_solid_kPa, porosity):
    """Calculate scaffold modulus from porosity."""
    relative_density = 1 - porosity
    E_scaffold = E_solid_kPa * (relative_density ** 2)
    return E_scaffold

def calculate_required_porosity(E_target_kPa, E_solid_kPa):
    """Calculate required porosity for target modulus."""
    relative_density = np.sqrt(E_target_kPa / E_solid_kPa)
    porosity = 1 - relative_density
    return porosity

# From Exercise 7: Rheology
def power_law_viscosity(shear_rate, K, n):
    """Calculate apparent viscosity."""
    return K * (shear_rate ** (n - 1))

def calculate_printability_number(yield_stress_Pa, density_kg_m3, height_mm):
    """Calculate shape fidelity metric."""
    g = 9.81
    h_m = height_mm * 1e-3
    Pr = yield_stress_Pa / (density_kg_m3 * g * h_m)
    return Pr

# Extrusion-specific calculations
def calculate_shear_rate_needle(flow_rate_uL_s, needle_diameter_mm):
    """Calculate wall shear rate in needle."""
    Q = flow_rate_uL_s * 1e-9  # m³/s
    R = (needle_diameter_mm / 2) * 1e-3  # m
    
    # For power-law fluid, approximate
    gamma_dot_wall = (4 * Q) / (np.pi * R**3)
    return gamma_dot_wall

def calculate_pressure_drop(viscosity_Pa_s, flow_rate_uL_s, 
                           needle_diameter_mm, needle_length_mm):
    """Calculate pressure drop for Newtonian fluid (approximate)."""
    Q = flow_rate_uL_s * 1e-9  # m³/s
    R = (needle_diameter_mm / 2) * 1e-3  # m
    L = needle_length_mm * 1e-3  # m
    
    # Hagen-Poiseuille (Newtonian approximation)
    dP = (8 * viscosity_Pa_s * L * Q) / (np.pi * R**4)
    return dP / 1000  # Convert to kPa

def calculate_residence_time(needle_length_mm, flow_rate_uL_s, needle_diameter_mm):
    """Calculate time cells spend in needle."""
    V_needle = np.pi * ((needle_diameter_mm/2)**2) * needle_length_mm  # µL
    t_residence = V_needle / flow_rate_uL_s  # seconds
    return t_residence

# Comprehensive assessment
def assess_bioprinting_design(design_params):
    """
    Comprehensive assessment of bioprinting design.
    
    Parameters:
    -----------
    design_params : dict
        Complete design specification
    
    Returns:
    --------
    assessment : dict
        Detailed assessment with scores and recommendations
    """
    assessment = {
        'overall_score': 0,
        'rating': '',
        'viability_score': 0,
        'mechanical_score': 0,
        'transport_score': 0,
        'printability_score': 0,
        'issues': [],
        'recommendations': [],
        'risks': []
    }
    
    # Extract parameters
    cell_viability = design_params.get('cell_viability', 0)
    E_scaffold = design_params.get('modulus', 0)
    E_target_min = design_params.get('target_E_min', 10)
    E_target_max = design_params.get('target_E_max', 50)
    pore_size = design_params.get('pore_size', 0)
    Pr = design_params.get('printability_number', 0)
    print_time = design_params.get('print_time', 0)
    
    # Score cell viability (40 points max)
    if cell_viability >= 85:
        assessment['viability_score'] = 40
    elif cell_viability >= 80:
        assessment['viability_score'] = 35
        assessment['issues'].append('Viability slightly below ideal (>85%)')
    elif cell_viability >= 70:
        assessment['viability_score'] = 25
        assessment['issues'].append('Viability concerning (<80%)')
        assessment['recommendations'].append('Reduce shear stress or printing time')
    else:
        assessment['viability_score'] = 10
        assessment['issues'].append('Poor cell viability (<70%)')
        assessment['risks'].append('High cell death rate')
    
    # Score mechanical properties (25 points max)
    if E_target_min <= E_scaffold <= E_target_max:
        assessment['mechanical_score'] = 25
    elif 0.5 * E_target_min <= E_scaffold <= 2 * E_target_max:
        assessment['mechanical_score'] = 15
        assessment['issues'].append('Modulus outside target range')
    else:
        assessment['mechanical_score'] = 5
        assessment['issues'].append('Modulus far from target')
        if E_scaffold < E_target_min:
            assessment['recommendations'].append('Increase polymer concentration or crosslinking')
        else:
            assessment['recommendations'].append('Decrease concentration or increase porosity')
    
    # Score transport/vascularization (20 points max)
    if pore_size >= 250:
        assessment['transport_score'] = 20
    elif pore_size >= 200:
        assessment['transport_score'] = 15
    elif pore_size >= 100:
        assessment['transport_score'] = 10
        assessment['issues'].append('Pore size marginal for vascularization')
    else:
        assessment['transport_score'] = 5
        assessment['issues'].append('Pores too small for vessel ingrowth')
        assessment['risks'].append('Center necrosis without perfusion')
    
    # Score printability (15 points max)
    if Pr > 5:
        assessment['printability_score'] = 15
    elif Pr > 2:
        assessment['printability_score'] = 10
    elif Pr > 1:
        assessment['printability_score'] = 5
        assessment['issues'].append('Marginal shape fidelity')
    else:
        assessment['printability_score'] = 2
        assessment['issues'].append('Poor shape retention - will collapse')
        assessment['recommendations'].append('Increase yield stress or add support bath')
    
    # Check print time
    if print_time <= 120:
        # Good - add bonus
        assessment['overall_score'] += 5
    elif print_time <= 180:
        # Acceptable
        pass
    else:
        assessment['issues'].append('Print time too long (>3 hours)')
        assessment['risks'].append('Cell damage from extended handling')
    
    # Calculate overall score
    assessment['overall_score'] += (assessment['viability_score'] +
                                   assessment['mechanical_score'] +
                                   assessment['transport_score'] +
                                   assessment['printability_score'])
    
    # Overall rating
    if assessment['overall_score'] >= 85:
        assessment['rating'] = 'Excellent'
    elif assessment['overall_score'] >= 70:
        assessment['rating'] = 'Good'
    elif assessment['overall_score'] >= 50:
        assessment['rating'] = 'Marginal'
    else:
        assessment['rating'] = 'Poor - Redesign Needed'
    
    return assessment

print("✓ Integrated design functions defined!")
print("\nReady to design complete bioprinting process!")

## Part 2: Complete Process Design Workflow

### 🎯 MAIN STUDENT TASK: Design Your Cardiac Tissue Bioprinting Process

**Work through each step systematically:**

In [ ]:
# =====================================================================
# STEP 1: DEFINE PROJECT REQUIREMENTS AND CONSTRAINTS
# =====================================================================

print("="*70)
print("STEP 1: PROJECT REQUIREMENTS")
print("="*70)

# Target tissue
tissue_type = 'cardiac'
construct_size = {'length': 10, 'width': 10, 'height': 5}  # mm

# Cell requirements
cell_types = {
    'cardiomyocytes': {
        'density': 30e6,  # cells/mL
        'viability_target': 80,  # %
        'shear_sensitivity': 'high',
        'max_shear': 5  # kPa
    },
    'endothelial': {
        'density': 10e6,  # cells/mL
        'viability_target': 85,  # %
        'shear_sensitivity': 'medium',
        'max_shear': 8  # kPa
    }
}

# Mechanical targets (native myocardium)
mechanical_targets = {
    'E_min': 10,  # kPa
    'E_max': 50,  # kPa
    'E_target': 30  # kPa (typical)
}

# Transport requirements
transport_requirements = {
    'min_pore_size': 200,  # µm (for vascularization)
    'target_pore_size': 300,  # µm
    'porosity_min': 0.6,  # 60%
}

# Process constraints
process_constraints = {
    'max_print_time': 120,  # minutes
    'max_pressure': 200,  # kPa
    'temperature_range': (15, 37),  # °C
    'max_shear_overall': 5  # kPa (limited by CMs)
}

print(f"\nTarget: {tissue_type.upper()} tissue")
print(f"Dimensions: {construct_size['length']}×{construct_size['width']}×{construct_size['height']} mm")
print(f"Volume: {construct_size['length']*construct_size['width']*construct_size['height']} mm³")
print(f"\nCell types: {list(cell_types.keys())}")
print(f"Mechanical target: {mechanical_targets['E_min']}-{mechanical_targets['E_max']} kPa")
print(f"Pore size target: >{transport_requirements['min_pore_size']} µm")
print(f"Max print time: {process_constraints['max_print_time']} min")
print("\n" + "="*70)

In [ ]:
# =====================================================================
# STEP 2: SELECT BIOPRINTING TECHNIQUE
# =====================================================================

print("\n" + "="*70)
print("STEP 2: TECHNIQUE SELECTION")
print("="*70)

techniques = {
    'extrusion': {
        'resolution': 100,  # µm
        'speed': 'slow',
        'cell_density': 'high',
        'shear_stress': 'medium-high',
        'suitable_for_cardiac': True,
        'pros': ['High cell density', 'Good for viscous bioinks', 'Established method'],
        'cons': ['Significant shear stress', 'Lower resolution', 'Slower']
    },
    'inkjet': {
        'resolution': 50,  # µm
        'speed': 'fast',
        'cell_density': 'low',
        'shear_stress': 'high',
        'suitable_for_cardiac': False,
        'pros': ['High resolution', 'Fast', 'Low cost'],
        'cons': ['Very high shear', 'Low viscosity only', 'Low cell density']
    },
    'light_based': {
        'resolution': 25,  # µm
        'speed': 'fast',
        'cell_density': 'medium',
        'shear_stress': 'none',
        'suitable_for_cardiac': True,
        'pros': ['No shear stress', 'High resolution', 'Fast'],
        'cons': ['Photodamage risk', 'Limited materials', 'Equipment cost']
    }
}

# Decision matrix
print("\nTechnique Comparison for Cardiac Tissue:")
print("-" * 70)
for tech_name, props in techniques.items():
    print(f"\n{tech_name.upper()}:")
    print(f"  Resolution: {props['resolution']} µm")
    print(f"  Cell density capability: {props['cell_density']}")
    print(f"  Shear stress: {props['shear_stress']}")
    print(f"  Suitable: {'✓' if props['suitable_for_cardiac'] else '✗'}")

# ==========================================
# STUDENT DECISION POINT
# ==========================================
selected_technique = 'extrusion'  # Options: 'extrusion', 'inkjet', 'light_based'
# ==========================================

print(f"\n🎯 SELECTED TECHNIQUE: {selected_technique.upper()}")
print(f"\nRationale for {selected_technique}:")
for pro in techniques[selected_technique]['pros']:
    print(f"  ✓ {pro}")
print(f"\nChallenges to address:")
for con in techniques[selected_technique]['cons']:
    print(f"  ⚠️  {con}")

print("\n" + "="*70)

In [ ]:
# =====================================================================
# STEP 3: DESIGN BIOINK FORMULATION
# =====================================================================

print("\n" + "="*70)
print("STEP 3: BIOINK FORMULATION")
print("="*70)

# Available materials database
materials_database = {
    'GelMA': {
        'E_solid': 100,  # kPa (after crosslinking)
        'K': 2.0,  # Pa·s^n (rheology)
        'n': 0.45,
        'tau_y_per_percent': 15,  # Pa per % w/v
        'biocompatibility': 'excellent',
        'crosslink': 'photo',
        'typical_range': (3, 10)  # % w/v
    },
    'alginate': {
        'E_solid': 60,  # kPa
        'K': 1.5,
        'n': 0.50,
        'tau_y_per_percent': 20,  # Pa per % w/v
        'biocompatibility': 'good',
        'crosslink': 'ionic',
        'typical_range': (1, 4)
    },
    'collagen': {
        'E_solid': 50,  # kPa
        'K': 0.5,
        'n': 0.55,
        'tau_y_per_percent': 8,
        'biocompatibility': 'excellent',
        'crosslink': 'thermal/enzymatic',
        'typical_range': (1, 5)
    },
    'fibrinogen': {
        'E_solid': 40,  # kPa
        'K': 0.3,
        'n': 0.60,
        'tau_y_per_percent': 5,
        'biocompatibility': 'excellent',
        'crosslink': 'enzymatic',
        'typical_range': (2, 10)
    }
}

# ==========================================
# STUDENT DESIGN DECISIONS
# ==========================================

# Select primary material
primary_material = 'GelMA'  # Choose from materials_database
primary_concentration = 5.0  # % w/v

# Optional: blend with secondary material
use_blend = True
secondary_material = 'alginate'
secondary_concentration = 2.0  # % w/v

# Crosslinking strategy
crosslink_method = 'photo'  # 'photo', 'ionic', 'thermal', 'dual'

# ==========================================

print(f"\nPrimary Material: {primary_material}")
print(f"  Concentration: {primary_concentration}% w/v")
print(f"  Properties: {materials_database[primary_material]}")

if use_blend:
    print(f"\nSecondary Material: {secondary_material}")
    print(f"  Concentration: {secondary_concentration}% w/v")
    print(f"  Purpose: Enhance mechanical/rheological properties")

print(f"\nCrosslinking: {crosslink_method}")

# Calculate blended properties (simple rule of mixtures)
if use_blend:
    total_conc = primary_concentration + secondary_concentration
    w1 = primary_concentration / total_conc
    w2 = secondary_concentration / total_conc
    
    mat1 = materials_database[primary_material]
    mat2 = materials_database[secondary_material]
    
    # Blend properties
    E_solid = w1 * mat1['E_solid'] + w2 * mat2['E_solid']
    K_blend = w1 * mat1['K'] + w2 * mat2['K']
    n_blend = w1 * mat1['n'] + w2 * mat2['n']
    tau_y = (primary_concentration * mat1['tau_y_per_percent'] + 
            secondary_concentration * mat2['tau_y_per_percent'])
else:
    mat1 = materials_database[primary_material]
    E_solid = mat1['E_solid']
    K_blend = mat1['K'] * (primary_concentration ** 1.5)  # Concentration scaling
    n_blend = mat1['n']
    tau_y = primary_concentration * mat1['tau_y_per_percent']

print(f"\nCalculated Bioink Properties:")
print(f"  E_solid (after crosslinking): {E_solid:.1f} kPa")
print(f"  Consistency index (K): {K_blend:.2f} Pa·s^n")
print(f"  Flow index (n): {n_blend:.2f}")
print(f"  Yield stress: {tau_y:.0f} Pa")

print("\n" + "="*70)

In [ ]:
# =====================================================================
# STEP 4: OPTIMIZE PRINTING PARAMETERS
# =====================================================================

print("\n" + "="*70)
print("STEP 4: PRINTING PARAMETERS")
print("="*70)

# ==========================================
# STUDENT DESIGN DECISIONS
# ==========================================

# Nozzle configuration
needle_diameter = 0.25  # mm (options: 0.15, 0.20, 0.25, 0.30, 0.40)
needle_length = 12.7    # mm (standard)

# Printing speed and flow
print_speed = 10        # mm/s (options: 5, 10, 15, 20)
layer_height = 0.20     # mm (typically 80% of needle diameter)

# Calculate required flow rate
# Flow rate Q = v × A_strand
strand_width = needle_diameter * 1.2  # slight spreading
strand_height = layer_height
A_strand = strand_width * strand_height  # mm²
flow_rate_uL_s = print_speed * A_strand  # µL/s

# Design structure
infill_pattern = 'rectilinear'  # 'rectilinear', 'honeycomb', 'gyroid'
infill_density = 25  # % (options: 20, 25, 30, 40)

# Calculate resulting porosity
target_porosity = 1 - (infill_density / 100)

# Temperature
print_temperature = 25  # °C

# ==========================================

print(f"\nNozzle Configuration:")
print(f"  Diameter: {needle_diameter} mm")
print(f"  Length: {needle_length} mm")

print(f"\nPrinting Parameters:")
print(f"  Speed: {print_speed} mm/s")
print(f"  Layer height: {layer_height} mm")
print(f"  Flow rate: {flow_rate_uL_s:.2f} µL/s")
print(f"  Temperature: {print_temperature}°C")

print(f"\nStructure Design:")
print(f"  Infill pattern: {infill_pattern}")
print(f"  Infill density: {infill_density}%")
print(f"  Target porosity: {target_porosity:.0%}")

# Calculate pore size (approximate)
# For rectilinear pattern
if infill_pattern == 'rectilinear':
    strand_spacing = strand_width / (infill_density / 100)
    pore_size_estimate = (strand_spacing - strand_width) * 1000  # µm
else:
    # Rough estimate for other patterns
    pore_size_estimate = 300 * (1 - infill_density/100)

print(f"  Estimated pore size: {pore_size_estimate:.0f} µm")

print("\n" + "="*70)

In [ ]:
# =====================================================================
# STEP 5: PREDICT PERFORMANCE METRICS
# =====================================================================

print("\n" + "="*70)
print("STEP 5: PERFORMANCE PREDICTION")
print("="*70)

# Calculate shear conditions
shear_rate_wall = calculate_shear_rate_needle(flow_rate_uL_s, needle_diameter)
viscosity_at_wall = power_law_viscosity(shear_rate_wall, K_blend, n_blend)
shear_stress_wall = viscosity_at_wall * shear_rate_wall  # Pa
shear_stress_kPa = shear_stress_wall / 1000

# Calculate pressure
pressure_drop_kPa = calculate_pressure_drop(viscosity_at_wall, flow_rate_uL_s,
                                           needle_diameter, needle_length)

# Calculate residence time
residence_time = calculate_residence_time(needle_length, flow_rate_uL_s, needle_diameter)

# Predict cell viability
cm_viability = predict_cell_viability_shear(shear_stress_kPa, residence_time, 'cardiomyocyte')
ec_viability = predict_cell_viability_shear(shear_stress_kPa, residence_time, 'endothelial')
overall_viability = (cm_viability + ec_viability) / 2

# Calculate mechanical properties
E_scaffold = gibson_ashby_modulus(E_solid, target_porosity)

# Calculate shape fidelity
bioink_density = 1050  # kg/m³ (typical hydrogel)
Pr = calculate_printability_number(tau_y, bioink_density, construct_size['height'])

# Estimate print time
# Approximate path length for rectilinear infill
volume = construct_size['length'] * construct_size['width'] * construct_size['height']  # mm³
volume_to_print = volume * (infill_density / 100)
path_length = volume_to_print / (strand_width * layer_height)  # mm
print_time = path_length / (print_speed * 60)  # minutes

# Gelation time
total_conc = primary_concentration if not use_blend else primary_concentration + secondary_concentration
gelation_time = calculate_gelation_time(print_temperature, total_conc, crosslink_method)

# Vascularization assessment
vasc_need = assess_vascularization_need(construct_size['height'], pore_size_estimate)

print("\n📊 PREDICTED PERFORMANCE:")
print("="*70)

print("\n1. CELL VIABILITY")
print(f"  Shear stress: {shear_stress_kPa:.2f} kPa")
print(f"  Residence time: {residence_time:.2f} s")
print(f"  Cardiomyocyte viability: {cm_viability:.1f}%")
print(f"  Endothelial viability: {ec_viability:.1f}%")
print(f"  Overall viability: {overall_viability:.1f}%")
if overall_viability >= 80:
    print("  ✅ MEETS REQUIREMENT (>80%)")
else:
    print("  ❌ BELOW REQUIREMENT")

print("\n2. MECHANICAL PROPERTIES")
print(f"  Scaffold modulus: {E_scaffold:.1f} kPa")
print(f"  Target range: {mechanical_targets['E_min']}-{mechanical_targets['E_max']} kPa")
if mechanical_targets['E_min'] <= E_scaffold <= mechanical_targets['E_max']:
    print("  ✅ WITHIN TARGET RANGE")
else:
    print("  ⚠️  OUTSIDE TARGET RANGE")

print("\n3. TRANSPORT & VASCULARIZATION")
print(f"  Porosity: {target_porosity:.0%}")
print(f"  Pore size: {pore_size_estimate:.0f} µm")
print(f"  Minimum for vessels: {transport_requirements['min_pore_size']} µm")
if pore_size_estimate >= transport_requirements['min_pore_size']:
    print("  ✅ ADEQUATE FOR VASCULARIZATION")
else:
    print("  ⚠️  MARGINAL FOR VASCULARIZATION")
print(f"  Vascularization need: {vasc_need['urgency']} ({vasc_need['reason']})")

print("\n4. SHAPE FIDELITY")
print(f"  Printability number (Pr): {Pr:.2f}")
if Pr > 5:
    print("  ✅ EXCELLENT SHAPE RETENTION")
elif Pr > 2:
    print("  ✓ GOOD SHAPE RETENTION")
elif Pr > 1:
    print("  ⚠️  MARGINAL STABILITY")
else:
    print("  ❌ WILL COLLAPSE")

print("\n5. PROCESS EFFICIENCY")
print(f"  Pressure drop: {pressure_drop_kPa:.1f} kPa")
print(f"  Maximum allowed: {process_constraints['max_pressure']} kPa")
if pressure_drop_kPa <= process_constraints['max_pressure']:
    print("  ✅ WITHIN PRESSURE LIMIT")
else:
    print("  ❌ EXCEEDS PRESSURE LIMIT")

print(f"  Estimated print time: {print_time:.1f} min")
print(f"  Maximum allowed: {process_constraints['max_print_time']} min")
if print_time <= process_constraints['max_print_time']:
    print("  ✅ WITHIN TIME LIMIT")
else:
    print("  ⚠️  EXCEEDS TIME TARGET")

print(f"  Gelation time: {gelation_time:.1f} s")

print("\n" + "="*70)

## Part 3: Comprehensive Assessment and Visualization

In [ ]:
# =====================================================================
# STEP 6: COMPREHENSIVE ASSESSMENT
# =====================================================================

# Compile all design parameters
design_params = {
    'cell_viability': overall_viability,
    'modulus': E_scaffold,
    'target_E_min': mechanical_targets['E_min'],
    'target_E_max': mechanical_targets['E_max'],
    'pore_size': pore_size_estimate,
    'printability_number': Pr,
    'print_time': print_time
}

# Get comprehensive assessment
assessment = assess_bioprinting_design(design_params)

# Create detailed visualization
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Overall Score', 'Category Breakdown',
                   'Cell Viability by Type', 'Mechanical Match',
                   'Print Timeline', 'Risk Assessment'),
    specs=[[{'type': 'indicator'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'table'}]],
    row_heights=[0.3, 0.35, 0.35]
)

# Plot 1: Overall score gauge
score = assessment['overall_score']
rating = assessment['rating']

if 'Excellent' in rating:
    gauge_color = 'green'
elif 'Good' in rating:
    gauge_color = 'lightgreen'
elif 'Marginal' in rating:
    gauge_color = 'yellow'
else:
    gauge_color = 'red'

fig.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=score,
        title={'text': f"Design Quality<br>{rating}"},
        delta={'reference': 80},
        gauge={
            'axis': {'range': [None, 105]},
            'bar': {'color': gauge_color},
            'steps': [
                {'range': [0, 50], 'color': 'lightgray'},
                {'range': [50, 70], 'color': 'lightyellow'},
                {'range': [70, 85], 'color': 'lightgreen'},
                {'range': [85, 105], 'color': 'green'}],
            'threshold': {
                'line': {'color': 'darkgreen', 'width': 4},
                'thickness': 0.75,
                'value': 85}
        }),
    row=1, col=1
)

# Plot 2: Category breakdown
categories = ['Cell\nViability', 'Mechanical\nProperties', 
             'Transport', 'Printability']
scores_cat = [assessment['viability_score'], assessment['mechanical_score'],
             assessment['transport_score'], assessment['printability_score']]
max_scores = [40, 25, 20, 15]

fig.add_trace(
    go.Bar(x=categories, y=scores_cat, name='Achieved',
          marker_color='lightblue',
          text=[f'{s}/{m}' for s, m in zip(scores_cat, max_scores)],
          textposition='outside'),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=categories, y=max_scores, name='Maximum',
          marker_color='lightgray', opacity=0.5),
    row=1, col=2
)
fig.update_yaxes(title_text="Score", row=1, col=2)

# Plot 3: Cell viability by type
fig.add_trace(
    go.Bar(x=['Cardiomyocytes', 'Endothelial Cells', 'Overall'],
          y=[cm_viability, ec_viability, overall_viability],
          marker_color=['#e74c3c', '#3498db', '#2ecc71'],
          text=[f'{v:.1f}%' for v in [cm_viability, ec_viability, overall_viability]],
          textposition='outside'),
    row=2, col=1
)
# Add threshold line manually for Plotly compatibility
fig.add_trace(
    go.Scatter(x=[0, 2], y=[80, 80],
              mode='lines', name='Target (80%)',
              line=dict(color='red', width=2, dash='dash'),
              showlegend=False),
    row=2, col=1
)
fig.update_yaxes(title_text="Viability (%)", range=[0, 105], row=2, col=1)

# Plot 4: Mechanical property match
E_range = np.linspace(0, 100, 100)
fig.add_trace(
    go.Scatter(x=E_range, y=[1]*len(E_range), fill='tozeroy',
              fillcolor='rgba(200,200,200,0.3)', line_width=0,
              showlegend=False, hoverinfo='skip'),
    row=2, col=2
)
# Target range - manual for compatibility
x_min = mechanical_targets['E_min']
x_max = mechanical_targets['E_max']
fig.add_trace(
    go.Scatter(x=[x_min, x_max, x_max, x_min, x_min],
              y=[0, 0, 1, 1, 0],
              fill='toself', fillcolor='rgba(0,255,0,0.3)',
              line=dict(width=0), showlegend=False, hoverinfo='skip'),
    row=2, col=2
)
# Achieved value - manual for compatibility  
fig.add_trace(
    go.Scatter(x=[E_scaffold, E_scaffold], y=[0, 1],
              mode='lines', line=dict(color='red', width=3),
              name=f'Achieved: {E_scaffold:.1f} kPa',
              showlegend=False),
    row=2, col=2
)
fig.update_xaxes(title_text="Elastic Modulus (kPa)", range=[0, 100], row=2, col=2)
fig.update_yaxes(visible=False, row=2, col=2)

# Plot 5: Print timeline
time_stages = ['Prep', 'Print', 'Crosslink', 'Maturation']
time_values = [10, print_time, gelation_time/60, 60]  # minutes
colors_time = ['lightblue', 'blue', 'orange', 'green']

# Cumulative for stacked timeline
cumulative_time = np.cumsum([0] + time_values[:-1])
for i, (stage, duration, cum_t, color) in enumerate(zip(time_stages, time_values, 
                                                         cumulative_time, colors_time)):
    fig.add_trace(
        go.Scatter(x=[cum_t, cum_t+duration], y=[1, 1],
                  mode='lines', line=dict(color=color, width=20),
                  name=stage, hovertemplate=f'{stage}: {duration:.1f} min'),
        row=3, col=1
    )
fig.update_xaxes(title_text="Time (minutes)", row=3, col=1)
fig.update_yaxes(visible=False, row=3, col=1)

# Plot 6: Risk summary table
risk_data = []
if assessment['issues']:
    for issue in assessment['issues'][:5]:  # Top 5 issues
        risk_data.append(['⚠️', issue[:40]])
if assessment['risks']:
    for risk in assessment['risks'][:3]:  # Top 3 risks
        risk_data.append(['❌', risk[:40]])

if not risk_data:
    risk_data = [['✅', 'No major issues identified']]

fig.add_trace(
    go.Table(
        header=dict(values=['Status', 'Issue/Risk'],
                   fill_color='paleturquoise',
                   align='left'),
        cells=dict(values=list(zip(*risk_data)),
                  fill_color='lavender',
                  align='left',
                  font=dict(size=10))),
    row=3, col=2
)

fig.update_layout(height=1100, showlegend=True,
                 title_text=f"Cardiac Tissue Bioprinting Design Assessment<br><sub>Overall Score: {score}/100 - {rating}</sub>")
fig.show()

print("\n" + "="*70)
print("FINAL ASSESSMENT")
print("="*70)
print(f"\nOverall Score: {score}/100")
print(f"Rating: {rating}")

if assessment['issues']:
    print(f"\n⚠️  Issues Identified ({len(assessment['issues'])})  :")
    for i, issue in enumerate(assessment['issues'], 1):
        print(f"  {i}. {issue}")

if assessment['recommendations']:
    print(f"\n💡 Recommendations ({len(assessment['recommendations'])}):")
    for i, rec in enumerate(assessment['recommendations'], 1):
        print(f"  {i}. {rec}")

if assessment['risks']:
    print(f"\n❌ Critical Risks ({len(assessment['risks'])}):")
    for i, risk in enumerate(assessment['risks'], 1):
        print(f"  {i}. {risk}")

print("\n" + "="*70)

## Part 4: Design Space Exploration and Optimization

In [ ]:
# Create design space exploration
print("\n" + "="*70)
print("DESIGN SPACE EXPLORATION")
print("="*70)

# Vary two key parameters
concentrations_space = np.linspace(2, 8, 30)
print_speeds_space = np.linspace(5, 20, 30)

C_grid, V_grid = np.meshgrid(concentrations_space, print_speeds_space)

# Calculate metrics across design space
viability_grid = np.zeros_like(C_grid)
modulus_grid = np.zeros_like(C_grid)
time_grid = np.zeros_like(C_grid)
score_grid = np.zeros_like(C_grid)

for i in range(C_grid.shape[0]):
    for j in range(C_grid.shape[1]):
        conc = C_grid[i, j]
        speed = V_grid[i, j]
        
        # Recalculate with new parameters
        K_temp = K_blend * (conc / primary_concentration) ** 1.5
        Q_temp = speed * strand_width * layer_height
        gamma_temp = calculate_shear_rate_needle(Q_temp, needle_diameter)
        eta_temp = power_law_viscosity(gamma_temp, K_temp, n_blend)
        tau_temp = eta_temp * gamma_temp / 1000  # kPa
        t_res_temp = calculate_residence_time(needle_length, Q_temp, needle_diameter)
        
        viab_temp = predict_cell_viability_shear(tau_temp, t_res_temp, 'cardiomyocyte')
        E_temp_solid = E_solid * (conc / primary_concentration)
        E_temp = gibson_ashby_modulus(E_temp_solid, target_porosity)
        time_temp = print_time * (print_speed / speed)
        
        # Score based on targets
        score_temp = 0
        if viab_temp >= 80:
            score_temp += 40
        elif viab_temp >= 70:
            score_temp += 20
        
        if mechanical_targets['E_min'] <= E_temp <= mechanical_targets['E_max']:
            score_temp += 30
        elif 0.5*mechanical_targets['E_min'] <= E_temp <= 2*mechanical_targets['E_max']:
            score_temp += 15
        
        if time_temp <= 120:
            score_temp += 20
        elif time_temp <= 180:
            score_temp += 10
        
        if tau_temp <= 5:
            score_temp += 10
        
        viability_grid[i, j] = viab_temp
        modulus_grid[i, j] = E_temp
        time_grid[i, j] = time_temp
        score_grid[i, j] = score_temp

# Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 14))

# Plot 1: Cell viability
contour1 = ax1.contourf(C_grid, V_grid, viability_grid, levels=20, cmap='RdYlGn')
contour1_lines = ax1.contour(C_grid, V_grid, viability_grid,
                             levels=[70, 80, 90], colors='black', linewidths=2)
ax1.clabel(contour1_lines, inline=True, fontsize=10, fmt='%d%%')
plt.colorbar(contour1, ax=ax1, label='Cell Viability (%)')

# Mark current design
ax1.plot(primary_concentration, print_speed, 'r*', markersize=20,
        markeredgecolor='black', markeredgewidth=2, label='Current Design')

ax1.set_xlabel('Concentration (% w/v)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Print Speed (mm/s)', fontsize=12, fontweight='bold')
ax1.set_title('Cell Viability Design Space', fontsize=13, fontweight='bold')
ax1.legend(loc='best')

# Plot 2: Mechanical properties
contour2 = ax2.contourf(C_grid, V_grid, modulus_grid, levels=20, cmap='viridis')
contour2_lines = ax2.contour(C_grid, V_grid, modulus_grid,
                             levels=[10, 30, 50], colors='white', linewidths=2)
ax2.clabel(contour2_lines, inline=True, fontsize=10, fmt='%d kPa')
plt.colorbar(contour2, ax=ax2, label='Elastic Modulus (kPa)')

# Mark target range
ax2.contour(C_grid, V_grid, modulus_grid,
           levels=[mechanical_targets['E_min'], mechanical_targets['E_max']],
           colors=['red', 'red'], linewidths=3, linestyles='dashed')

ax2.plot(primary_concentration, print_speed, 'r*', markersize=20,
        markeredgecolor='black', markeredgewidth=2)

ax2.set_xlabel('Concentration (% w/v)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Print Speed (mm/s)', fontsize=12, fontweight='bold')
ax2.set_title('Mechanical Properties', fontsize=13, fontweight='bold')

# Plot 3: Print time
contour3 = ax3.contourf(C_grid, V_grid, time_grid, levels=20, cmap='plasma')
contour3_lines = ax3.contour(C_grid, V_grid, time_grid,
                             levels=[60, 120, 180], colors='white', linewidths=2)
ax3.clabel(contour3_lines, inline=True, fontsize=10, fmt='%d min')
plt.colorbar(contour3, ax=ax3, label='Print Time (min)')

ax3.plot(primary_concentration, print_speed, 'r*', markersize=20,
        markeredgecolor='black', markeredgewidth=2)

ax3.set_xlabel('Concentration (% w/v)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Print Speed (mm/s)', fontsize=12, fontweight='bold')
ax3.set_title('Print Time', fontsize=13, fontweight='bold')

# Plot 4: Overall score
contour4 = ax4.contourf(C_grid, V_grid, score_grid, levels=20, cmap='RdYlGn')
contour4_lines = ax4.contour(C_grid, V_grid, score_grid,
                             levels=[50, 70, 85], colors='black', linewidths=2)
ax4.clabel(contour4_lines, inline=True, fontsize=10, fmt='%d')
plt.colorbar(contour4, ax=ax4, label='Overall Score')

# Find optimal point
opt_idx = np.unravel_index(score_grid.argmax(), score_grid.shape)
opt_conc = C_grid[opt_idx]
opt_speed = V_grid[opt_idx]
opt_score = score_grid[opt_idx]

ax4.plot(primary_concentration, print_speed, 'r*', markersize=20,
        markeredgecolor='black', markeredgewidth=2, label='Current Design')
ax4.plot(opt_conc, opt_speed, 'g*', markersize=20,
        markeredgecolor='black', markeredgewidth=2, label=f'Optimal (Score={opt_score:.0f})')

ax4.set_xlabel('Concentration (% w/v)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Print Speed (mm/s)', fontsize=12, fontweight='bold')
ax4.set_title('Overall Design Score', fontsize=13, fontweight='bold')
ax4.legend(loc='best')

plt.tight_layout()
plt.show()

print(f"\nDesign Space Analysis:")
print(f"  Current design score: {score:.0f}/100")
print(f"  Optimal design found:")
print(f"    - Concentration: {opt_conc:.1f}% w/v")
print(f"    - Print speed: {opt_speed:.1f} mm/s")
print(f"    - Predicted score: {opt_score:.0f}/100")
print(f"\n  Improvement potential: {opt_score - score:.0f} points")

print("\n" + "="*70)

## Part 5: Summary and Final Recommendations

In [ ]:
print("="*80)
print("EXERCISE 8: INTEGRATED BIOPRINTING - KEY TAKEAWAYS")
print("="*80)
print("""
🎯 CAPSTONE INSIGHTS: INTEGRATED BIOPRINTING DESIGN

1. DESIGN IS INHERENTLY MULTIVARIABLE
   → Must balance 4+ competing requirements simultaneously
   → Cell viability, mechanics, transport, printability all interact
   → No "perfect" solution - always trade-offs
   → Optimization requires systematic approach

2. CELL VIABILITY OFTEN THE LIMITING FACTOR
   → Shear stress in needle: τ = η·γ̇
   → Cardiomyocytes very sensitive (<5 kPa)
   → Limits: flow rate, viscosity, needle size
   → Strategies: larger needles, slower printing, shear-thinning bioinks

3. MECHANICAL-BIOLOGICAL TRADE-OFF
   → Higher concentration → higher modulus → higher shear stress
   → Higher porosity → better transport → lower modulus
   → Must find sweet spot for each tissue type
   → Cardiac: ~5% GelMA with 25-30% infill typical

4. VASCULARIZATION IS CRITICAL
   → Constructs >2 mm need vascular support
   → Requires: pores >200 µm + perfusion or pre-vascularization
   → Cannot rely on diffusion alone
   → Design for vessel ingrowth from start

5. SHAPE FIDELITY NEEDS BALANCE
   → Pr = τ_y/(ρgh) should be >2-5
   → Too high yield stress → hard to extrude
   → Too low → structure collapses
   → Fast gelation helps but must not damage cells

6. PROCESS TIMING MATTERS
   → Print time affects cell viability (handling stress)
   → Gelation must occur before next layer
   → Total time includes: prep, print, crosslink, maturation
   → Longer print → more variables to control

7. MULTI-MATERIAL STRATEGIES
   → Blending improves properties
   → Example: GelMA (photoclick) + alginate (ionic) = dual crosslinking
   → Can separate structural and biological functions
   → Adds complexity but enables better optimization

8. DESIGN SPACE EXPLORATION IS ESSENTIAL
   → Don't rely on single design point
   → Map out viable parameter ranges
   → Identify optimal combinations
   → Understand sensitivity to variations

9. FAILURE MODE ANALYSIS
   → Common failure modes:
     • High cell death (excessive shear/pressure)
     • Collapse (insufficient yield stress)
     • Center necrosis (insufficient transport)
     • Poor integration (inadequate gelation)
     • Dimensional inaccuracy (wrong parameters)
   → Identify and mitigate before printing

10. ITERATIVE REFINEMENT PROCESS
    → Initial design rarely optimal
    → Systematic optimization workflow:
      1. Define requirements
      2. Select technique
      3. Design initial formulation
      4. Predict performance
      5. Identify issues
      6. Optimize parameters
      7. Validate experimentally
      8. Iterate

PRACTICAL DESIGN GUIDELINES FOR CARDIAC TISSUE:

✓ Bioink: 4-6% GelMA + 1-2% alginate
✓ Nozzle: 0.25-0.30 mm (balance resolution vs shear)
✓ Speed: 8-12 mm/s (fast enough but not excessive shear)
✓ Infill: 25-30% (good porosity + adequate strength)
✓ Layer height: 0.20 mm (80% of nozzle diameter)
✓ Crosslinking: Dual (photo + ionic) for fast gelation
✓ Temperature: 20-25°C (warm enough for cell viability)
✓ Post-print: Perfusion or vascular co-culture immediately

NEXT STEPS AFTER DESIGN:

1. Experimental validation of rheology
2. Print test geometries (cubes, cylinders)
3. Measure cell viability at 4, 24, 72 hours
4. Characterize mechanical properties
5. Assess vascularization potential
6. Refine based on results
7. Scale to final construct

Remember: Computational design predicts, but experiments validate!
""")
print("="*80)

## 🎓 CAPSTONE REFLECTION QUESTIONS

### Question 1: Trade-off Analysis
**Your design has 75% cell viability but excellent mechanical properties. The supervising PI wants >85% viability. What three specific changes would you make to the bioink formulation and printing parameters? Justify each change with calculations.**

### Question 2: Failure Mode Diagnosis
**After printing, the construct's center shows 40% cell death at 48 hours while edges are fine. The structure itself is stable. Diagnose the problem and propose two solutions with quantitative justification.**

### Question 3: Scale-Up Challenge
**Your 5mm thick construct works well. The clinical application requires 15mm thickness. What fundamental problems arise? How would you redesign the entire process? Consider ALL aspects: bioink, printing, vascularization, perfusion.**

### Question 4: Alternative Technique Comparison
**Compare your extrusion-based design to a light-based (DLP) approach for the same cardiac construct. Calculate: (a) expected cell viability, (b) print time, (c) resolution, (d) mechanical properties. Which is better and why?**

### Question 5: Regulatory Strategy
**For FDA/CE approval, you need reproducible constructs with <10% variation in all properties. Identify the 3 most critical parameters to control and specify tolerance limits with justification. How would you validate manufacturing consistency?**

## 📚 FINAL PROJECT: Design Your Own Construct

### Extended Capstone Project

**Choose ONE of the following clinical applications:**

1. **Liver tissue patch** (20×20×2 mm)
   - Hepatocytes + endothelial cells
   - Requirements: High cell density (100M/mL), metabolic function, vascularization
   
2. **Cartilage implant** (15×15×3 mm)
   - Chondrocytes
   - Requirements: High modulus (E=5-20 MPa), no vascularization, shape fidelity
   
3. **Vascular graft** (diameter 4 mm, length 30 mm)
   - Smooth muscle cells + endothelial cells
   - Requirements: Burst pressure >200 mmHg, compliance, endothelialization
   
4. **Neural tissue** (10×10×1 mm)
   - Neurons + glial cells
   - Requirements: Ultra-low stiffness (E<1 kPa), aligned architecture, minimal shear

**Deliverables:**
1. Complete design specification (materials, parameters, technique)
2. Predicted performance metrics with calculations
3. Risk assessment and mitigation strategies
4. Validation plan (what experiments needed)
5. Timeline and cost estimate

**Use this notebook as template and modify parameters accordingly!**

## 🎯 Congratulations!

You've completed Exercise 8: Integrated Bioprinting Process Design!

**This capstone exercise integrated ALL concepts from Chapter 4:**
- ✓ Cell viability prediction under shear stress
- ✓ Mass transport and oxygen diffusion
- ✓ Gelation kinetics and crosslinking
- ✓ Photopolymerization physics
- ✓ Scaffold architecture and porosity
- ✓ Rheology and printability
- ✓ Multi-objective optimization
- ✓ Design space exploration

**You can now:**
- ✓ Design complete bioprinting processes from scratch
- ✓ Balance competing requirements systematically
- ✓ Predict performance before experimental validation
- ✓ Identify and mitigate failure modes
- ✓ Optimize parameters across design space
- ✓ Think critically about clinical translation

**Next Steps:**
- Apply these principles to your own research projects
- Validate predictions experimentally
- Explore advanced topics: multi-material printing, 4D printing, in-situ bioprinting
- Consider regulatory pathways for clinical translation

---

*Capstone Exercise for Biofabrication Chapter 4 - Master's Level Bioengineering*

**🎓 You are now ready to design and execute sophisticated bioprinting projects!**